# Indeed.com job portal scrapping using BeautifulSoup and regex package
By Group: Shreyash, Siddharth, Shubham , Likhit

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
import csv
from datetime import datetime
import re

In [3]:
url = "https://in.indeed.com/jobs?q={}&l={}"
r = requests.get(url)
htmlContent = r.content
#  print(htmlContent)

In [4]:
def get_url(position,location):
    url = "https://in.indeed.com/jobs?q={}&l={}"
    url1 = url.format(position,location)
    return url1
    

In [5]:
url = get_url("web developer","Navi Mumbai Maharashtra")

# Extracting raw Html

In [6]:
response = requests.get(url)

In [7]:
response

<Response [200]>

In [8]:
response.reason

''

In [9]:
soup = BeautifulSoup(response.text,'html.parser')

In [11]:
cards = soup.find_all('div',re.compile(r'slider_container'))
cards

[<div class="slider_container"><div class="slider_list"><div class="slider_item"><div class="job_seen_beacon"><table cellpadding="0" cellspacing="0" class="jobCard_mainContent" role="presentation"><tbody><tr><td class="resultContent"><div class="heading4 color-text-primary singleLineTitle tapItem-gutter"><h2 class="jobTitle jobTitle-color-purple jobTitle-newJob"><div class="new topLeft holisticNewBlue desktop"><span class="label">new</span></div><span title="Project Manager- Web/Software Development">Project Manager- Web/Software Development</span></h2></div><div class="heading6 company_location tapItem-gutter"><pre><span class="companyName">Spiralyze</span><div class="companyLocation">Mumbai, Maharashtra<span class="remote-bullet">•</span><span>Remote</span></div></pre></div><div class="heading6 tapItem-gutter metadataContainer"><div class="metadata salary-snippet-container"><span aria-label="₹30,000 to ₹80,000 a month" class="salary-snippet">₹30,000 - ₹80,000 a month</span></div></di

In [12]:
len(cards)

15

# prototype the model with a single record

In [13]:
card = cards[0]

In [14]:
card.div.span


<span class="label">new</span>

In [15]:
title = card.find('div', class_= re.compile(r'heading+')).text
title

'newProject Manager- Web/Software Development'

In [17]:
card['title'] = title

In [18]:
card['title']

'newProject Manager- Web/Software Development'

In [26]:
job_title = card['title']
job_title

'Full Stack Developer'

In [19]:
company = card.find('span',re.compile(r'companyName')).text
company

'Spiralyze'

In [20]:
job_location= card.find('div',class_= re.compile(r'companyLocation')).text
job_location

'Mumbai, Maharashtra•Remote'

In [21]:
job_summary = card.find('div',re.compile(r'job-snippet')).text
job_summary.strip()
job_sum = re.split('\n',job_summary)
job_sum

['',
 'We code those designs into challenger web pages.',
 'Coordination. Running big A/B tests requires teams of 5-10 people (researchers, strategists, designers,…',
 '']

In [22]:
date_job = card.find('span','date').text
date_job

'3 days ago'

In [23]:
today = datetime.today().strftime('%Y-%m-%d')
today

'2021-10-18'

In [24]:
job_salary = card.find('span',re.compile(r'salary-snippet')).text
job_salary

'₹30,000 - ₹80,000 a month'

In [25]:

try:
    job_salary = card.find('span',re.compile(r'salary-snippet')).text
except AttributeError:
    job_salary = ' '

# Generalize the model with the function

In [26]:
def get_record(card):
    card.div.span
    title = card.find('div', class_= re.compile(r'heading+')).text
    card['title'] = title
    job_title = card['title']
    company = card.find('span',re.compile(r'companyName')).text
    job_location= card.find('div',class_= re.compile(r'companyLocation')).text
    job_summary = card.find('div',re.compile(r'job-snippet')).text
    job_sum = re.split('\n',job_summary)
    date_job = card.find('span','date').text
    today = datetime.today().strftime('%Y-%m-%d')
    try:
        job_salary = card.find('span',re.compile(r'salary-snippet')).text
    except AttributeError:
        job_salary = ' '
    record = (job_title,company,job_location,job_sum,date_job,job_salary,today)
    return record 

In [27]:
records = []

for card in cards:
    record = get_record(card)
    records.append(record)

In [29]:
records[0]

('newProject Manager- Web/Software Development',
 'Spiralyze',
 'Mumbai, Maharashtra•Remote',
 ['',
  'We code those designs into challenger web pages.',
  'Coordination. Running big A/B tests requires teams of 5-10 people (researchers, strategists, designers,…',
  ''],
 '3 days ago',
 '₹30,000 - ₹80,000 a month',
 '2021-10-18')

In [30]:
records[1]

('Senior Web Developer',
 'Tonic Worldwide',
 'Mumbai, Maharashtra•Remote',
 ['',
  'Develop & implement modern web services using service-oriented architecture, RESTful APIs, and SOAP.',
  'Test, troubleshoot, and optimize application components for…',
  ''],
 '30+ days ago',
 '₹50,000 - ₹65,000 a month',
 '2021-10-18')

# Getting the next page

In [31]:
url ='https://in.indeed.com' + soup.find('a',{'aria-label': 'Next'}).get('href')
url

'https://in.indeed.com/jobs?q=web+developer&l=Navi+Mumbai+Maharashtra&start=10'

In [32]:
i = 0 
for i in range(0,3):
    
    try:
        url ='https://in.indeed.com' + soup.find('a',{'aria-label': 'Next'}).get('href')
    except AttributeError:
        break
        
        response = requests.get(url)
        soup = BeautifulSoup(response.text,'html.parser')
        cards = soup.find_all('div',re.compile(r'slider_container'))
        
        for card in cards:
            record = get_record(card)
            records.append(record)

In [33]:
len(records)

15

# Putting it all together

In [34]:
import csv
from datetime import datetime
import re
import requests
from bs4 import BeautifulSoup

def get_url(position,location):
    url = "https://in.indeed.com/jobs?q={}&l={}"
    url1 = url.format(position,location)
    return url1

def get_record(card):
    card.div.span
    title = card.find('div', class_= re.compile(r'heading+')).text
    card['title'] = title
    job_title = card['title']
    company = card.find('span',re.compile(r'companyName')).text
    job_location= card.find('div',class_= re.compile(r'companyLocation')).text
    job_summary = card.find('div',re.compile(r'job-snippet')).text
    job_sum = re.split('\n',job_summary)
    date_job = card.find('span','date').text
    today = datetime.today().strftime('%Y-%m-%d')
    try:
        job_salary = card.find('span',re.compile(r'salary-snippet')).text
    except AttributeError:
        job_salary = ' '
    record = (job_title,company,job_location,job_sum,date_job,job_salary,today)
    return record 

def main(position,location):
    records = []
    url = get_url(position,location)
    
    i = 0 
for i in range(0,3):
    
    try:
        url ='https://in.indeed.com' + soup.find('a',{'aria-label': 'Next'}).get('href')
    except AttributeError:
        break
        
        response = requests.get(url)
        soup = BeautifulSoup(response.text,'html.parser')
        cards = soup.find_all('div',re.compile(r'slider_container'))
        
        for card in cards:
            record = get_record(card)
            records.append(record)
    
    with open('result.csv','w',newline='',encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['JobTitle','Company','Location','Summary','ExtractDate','Salary',
                         'Today_Date'])
        writer.writerows(records)

In [35]:
main('web developer','Navi Mumbai, Maharashtra')

In [36]:
import pandas as pd

In [37]:
address = 'result.csv'
data = pd.read_csv(address)
data

,JobTitle,Company,Location,Summary,ExtractDate,Salary,Today_Date
0,newProject Manager- Web/Software Development,Spiralyze,"Mumbai, Maharashtra•Remote","['', 'We code those designs into challenger we...",3 days ago,"₹30,000 - ₹80,000 a month",2021-10-18
1,Senior Web Developer,Tonic Worldwide,"Mumbai, Maharashtra•Remote","['', 'Develop & implement modern web services ...",30+ days ago,"₹50,000 - ₹65,000 a month",2021-10-18
2,newWeb Designer & Developer,Ardent Research,"Mumbai, Maharashtra•Remote","['', 'Writes/reads clean well-documented custo...",Just posted,"₹9,676 - ₹18,866 a month",2021-10-18
3,newWeb Developer,Mobavenue Media Pvt Ltd,"Mumbai, Maharashtra•Remote","['', 'A solid understanding of how web applica...",Just posted,"₹18,000 - ₹20,000 a month",2021-10-18
4,newWeb Developer - PHP,Bambu,"Navi Mumbai, Maharashtra","['', 'Solid ability to translate UI/UX design ...",Just posted,,2021-10-18
5,Front-End Web Developer,TIC,"Navi Mumbai, Maharashtra","['', 'Proficient understanding of web markup, ...",30+ days ago,,2021-10-18
6,Web Developer,PTS Systems & Solutions Pvt Ltd,"Navi Mumbai, Maharashtra","['', 'Writing well designed, testable, efficie...",30+ days ago,"₹15,000 - ₹20,000 a month",2021-10-18
7,HTML Developer,G-ORDINATEUR PVT LTD,"Navi Mumbai, Maharashtra•Temporarily Remote","['', 'Experience building large consumer web a...",27 days ago,"₹1,20,000 - ₹2,40,000 a year",2021-10-18
8,Web Developer,GIT Technologies,"Vashi, Navi Mumbai, Maharashtra","['', 'Skills At least 1 - 2 years of experienc...",19 days ago,,2021-10-18
9,newWordpress Developer,Brain Candy,"Navi Mumbai, Maharashtra","['', 'Intimate understanding of the WordPress ...",4 days ago,,2021-10-18
